In [1]:
from llm import chat_openai
import numpy as np 
import json
import logging 
import matplotlib.pyplot as plt
import os
import openai
import re
import subprocess
from pathlib import Path
import shutil
import time 

In [2]:
goal = '''
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

Submission File Format
The file should contain a header and have the following format:

Id,SalePrice
1461,169000.1
1462,187724.1233
1463,175221
etc.
'''

In [3]:
exploration_progress = "None"

In [17]:
class CurriculumAgent():
    def __init__(self, goal="", memory=[], completed_tasks=[], failed_tasks=[], files="", saved_notes=""):
        self.goal = goal
        self.memory = memory
        self.completed_tasks = completed_tasks
        self.failed_tasks = failed_tasks
        self.files = files
        self.saved_notes = saved_notes

        # The crux is a Q&A process
        # Problem with this approach is you still have to deal with searching multiple times, and continuing to search or not. Approach: Or maybe if you search and you don't have the answer, that's a bad thing to search and you need to go more specific / ask a different question!
        self.system_prompt_automatic_curriculum = f'''You are a helpful assistant that asks questions to help me decide the next immediate question to answer on the computer. My ultimate goal is to discover as many useful pieces of information as possible, answer as many questions as possible, and become the best researcher in the world.

        Goal: {self.goal}

        I will give you the following information:
        Memory (oldest to newest): ...
        Recent links: ...
        Root folder inventory: ...
        Saved notes: ...
        Completed tasks so far: ...
        Failed tasks that are too hard: ...

        1) You should act as a mentor and guide me to the next question based on my current learning progress.
        2) Please be very specific about what question I need to answer.
        3) The next question should follow a clear format, such as "What do other papers say about [topic]", "What are the problems with [approach]", "How can I solve this [problem]", "Can results from [topic 1] be applied to [topic 2]?", "What are the similarities between [topic 1] for success and [topic 2]" , "What's significant about this paper: [paper]?", "What does [topic] mean?", etc. It should be a single question to collect useful information on. Do not propose multiple questions at the same time. Do not mention anything else. 
        4) The next question should not be too hard since the internet and I may not contain the full answer in a single article or have learned enough information to complete it yet. 
        5) The next question should be novel and interesting based on my current learning progress. I should look for rare and potentially useful pieces of information, upgrade my saved notes using better information, and discover new things. I should not be doing the same thing over and over again.
        6) I may sometimes need to repeat some questions or variations of the question if I need to collect more information to answer more difficult question. Only repeat questions if necessary. 
        7) I want to explore the world and discover new things. I don’t want to stay in my current state for too long. 
        8) Questions that require information beyond another person's ability to theoretically verify and reason if completed or correct should be avoided. For instance, "what else is there on the website?" and "what images and tables are on the website" are not ideal since they require visual confirmation from the screen. All the testing, coding, and asking other people questions should be avoided. Do not propose a question with these keywords. You should only respond in the format as described below:
        
        RESPONSE FORMAT: 
        Reasoning: Based on the information I listed above, do reasoning about what the next question should be. 
        Question: The next question. 
        
        Here’s an example response: 
        Reasoning: We know the we have a sword and we know there's fire, and fire lights things on fire. Therefore, we could try to make a firesword.
        Question: Could we make a firesword?
'''

        # TODO: This is optional, might be useful, but to focus on a system prompt of asking questions and answering questions.
        # System 2: this is a more scoped down version where we have the focus be on only answering questions -- reading and analyzing information & asking questions. No action items. 
        # The current above system 1 is better for self-driving labs type of work where there are going to be more tasks.

    def get_exploration_progress(self, completed_tasks, failed_tasks):
        # TODO: this should contain inventory of where we're at now and what files we have / memory stream
        return '''Completed tasks: None, Failed tasks: None'''

    def propose_next_question(self, skills, exploration_progress):
        '''
        This function decomposes a goal into tasks
        '''
#         gen_curriculum = f'''You are a first-rate mentor. 
        
#         Your ultimate goal is to teach someone how to win the entire competition. Your task is to propose the next best action to achieve the goal. Please first reason on how to do this, and then conclude with the next best action in this format: "next action: <insert next action>".

#         Rules:
#         - In the concluding next action, pretend that the reader can only see that next best action text so make sure the next action best is clear.
#         - Make sure that the next best action requires is a small step and reasonable enough for a high schooler to figure out how to do it.

#         Exploration progress:
#         1. {exploration_progress}
        
#         Goal:
#         {self.goal}

#         Skills that you can use:
#         1. think(): to think about what to do next
#         2. read(file): to read a file, but only the first 4000 chars
#         3. write(file): to write a file
#         4. search(query): to google search a query and get back the first search result

#         Example of a good task: Look at the data and note down observations.
#         Why it’s good: Because the capabilities enabled for the agent are available, specifically look() and think().
        
#         Example of a bad task: Perform EDA on the dataset.
#         Why it’s bad: Because coding is not a capability that is enabled for the agent. 
# '''
        
        user_prompt = ""
        observation = {
            "memory": f"Memory: {self.memory}\n\n",
            "root_folder_inventory": f"Root folder inventory: {self.files}\n\n",
            "saved_notes": f"Saved notes: {self.saved_notes}\n\n",
            "completed_tasks": f"Completed tasks so far: {self.completed_tasks}\n\n",
            "failed_tasks": f"Failed tasks that are too hard: {self.failed_tasks}\n\n",
        }
        user_prompt += "".join(observation.values())
        
        print("System prompt for generating curriculum: \n", self.system_prompt_automatic_curriculum, "\n User prompt: ", user_prompt)
        next_question_response = chat_openai(user_prompt, system_prompt=self.system_prompt_automatic_curriculum)[0]
        print("Response: ", next_question_response)
        next_question = self.parse_message(next_question_response)["next_question"]
        return next_question
    
    def get_completed_tasks():
        pass

    def get_failed_tasks():
        pass

    def parse_message(self, message):
        question = ""
        for line in message.split("\n"):
            if line.startswith("Question:"):
                question = line[9:].strip()
        assert question, "Question not found in Curriculum Agent response"
        return {"next_question": question}

files = '''train.csv - the training set
test.csv - the test set
data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
data_fields.txt - a brief version of what you'll find in the data description file.
'''
curriculum_agent = CurriculumAgent(goal=goal, files=files)

In [18]:
next_question = curriculum_agent.propose_next_question([], [])

System prompt for generating curriculum: 
 You are a helpful assistant that asks questions to help me decide the next immediate question to answer on the computer. My ultimate goal is to discover as many useful pieces of information as possible, answer as many questions as possible, and become the best researcher in the world.

        Goal: 
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric
Submissions are ev

In [16]:
next_question

'What is the structure and content of the dataset in the train.csv file?'

In [12]:
task = ""
for line in next_question.split("\n"):
    if line.startswith("Question:"):
        task = line[9:].strip()
assert task, "Task not found in Curriculum Agent response"
return {"next_task": task}

{'next_task': 'What is the structure and content of the dataset in the train.csv file?'}


In [12]:
next_task = next_task.split("Next action: ")[1]

In [13]:
next_task

'Start by exploring the dataset and understanding the variables and their relationships with the target variable (SalePrice).'

In [16]:
def generate_how(goal, next_task, task_history = []):
    '''
    This function creates the function for the task and "how" for the task
    '''
    gen_how_prompt = f'''You are a first-rate problem solver. 
    
    Your task is to operationalize the following action item as a code function: {next_task}
    Only use action functions that you already have and other normal code. You must solve the problem using code, action functions that you create and can build upon, and words.

    Write a function for how to achieve this task using the action functions that you have. If you cannot achieve this task with the action functions that you have, then respond with "I don't know because <your reasoning>". If you can achieve this task, then please first reason on how to do this, and then conclude with the task function in this format: "Task function: <insert task function name>(<insert args>): <insert action functions to use>".

    Rules:
    - In the concluding list of steps, pretend that the reader can only see the list of steps and not your reasoning so make sure the list of steps is clear.
    - Make sure that the next best action requires is a small step and reasonable enough for a high schooler to figure out how to do it.

    Action functions that you have:
    1) think(prompt) -- thinking out loud given the prompt
    2) web_search(query) -- web searching for query and returning some information
    3) read(file) -- reading file, but the first 4000 chars
    4) write(file) -- writing to file

    Action functions that you are constrained to currently not have:
    1) code() -- writing code
    2) see() -- look at visualizations

    Current inventory:
    File descriptions
    train.csv - the training set
    test.csv - the test set
    data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
    sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
    data_fields.txt - a brief version of what you'll find in the data description file.
    '''
    print("Prompt for generating how to execute: \n", gen_how_prompt)
    next_step = chat_openai(gen_how_prompt)[0]
    return next_step

gen_function = generate_how(goal, next_task, [])
print(gen_function)

Prompt for generating how to execute: 
 You are a first-rate problem solver. 
    
    Your task is to operationalize the following action item as a code function: To achieve the goal of predicting the sales price for each house in the competition, we need to build a predictive model using the given dataset. Here is the next best action:

Next action: Preprocess the data by handling missing values, encoding categorical variables, and scaling numerical variables.

Explanation: Preprocessing the data is an essential step in building a predictive model. Handling missing values involves imputing or removing missing data points. Encoding categorical variables converts them into numerical representations that can be used by the model. Scaling numerical variables ensures that they are on a similar scale, preventing any variable from dominating the model's learning process.

By completing this next action, you will have prepared the dataset for model training and prediction.
    Only use actio